In [27]:
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
from dashgrid.dgrid_components import PlotlyCandles as plc
from dashgrid import db_info as dbi

import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output,State
from dash.exceptions import PreventUpdate
import dash_table
import pandas as pd
import numpy as np
import json
import logging
import datetime
import functools
import random
import inspect
from pandasql import sqldf
import datetime,base64,io,pytz


In [28]:
DEFAULT_LOG_PATH = './logfile.log'
DEFAULT_LOG_LEVEL = 'INFO'

def init_root_logger(logfile=DEFAULT_LOG_PATH,logging_level=DEFAULT_LOG_LEVEL):
    level = logging_level
    if level is None:
        level = logging.DEBUG
    # get root level logger
    logger = logging.getLogger()
    if len(logger.handlers)>0:
        return logger
    logger.setLevel(logging.getLevelName(level))

    fh = logging.FileHandler(logfile)
    fh.setLevel(logging.DEBUG)
    # create console handler with a higher log level
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter and add it to the handlers
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    # add the handlers to the logger
    logger.addHandler(fh)
    logger.addHandler(ch)   
    return logger


In [29]:
logger = init_root_logger(logging_level='DEBUG')


In [30]:
def stop_callback(errmess,logger=None):
    m = "****************************** " + errmess + " ***************************************"     
    if logger is not None:
        logger.debug(m)
    raise PreventUpdate()


# Dlink: Link Dash components 

In [31]:
def slice_df(df_in,input_list):
    """!
        ### slice_df slices a DataFrame according to a list of values.
        1. The arg ```input_list``` contains values that coincide with columns of the DataFrame
        2. The method returns a new filtered DataFrame where:
         * the method filters each column using the respective value in ```input_list```
         * if there is a ```None``` in any item in the input list, the method ignores filtering on that column
        :param df_in: DataFrame to be sliced
        :param input_list: list of values to use as criteria for each column slice. 
                            A value of None will cause the method to not slice that column
        :return: new DataFrame that has been sliced
    """
    dfc = df_in.drop_duplicates().copy()
    cols = dfc.columns.values
    current_input = 0
    for i in range(len(input_list)):
        il = input_list[i]
        col = cols[i]
        if il is not None:
            try:                
                dfc = dfc[dfc[col]==il]
            except Exception as e:
                raise ValueError(e)
        current_input += 1
    return dfc


In [32]:
def choices_from_df(df_in,input_list,col_to_select):
    '''
    Create a list of dicts that conform to an options list for a Dash componet like RadioItems or Dropdown,where:
    1. Each dict in the list has a 'label' key, and a 'value' key
    2. ```df_in``` is sliced
    '''
    dfc = slice_df(df_in,input_list)
    if dfc is None or len(dfc)<1:
        return None,None
    unique_values = dfc[col_to_select].unique()
    choices = unique_values[0],[{'label':uv,'value':uv} for uv in unique_values]
    return choices



In [33]:
GRID_STYLE = {'display': 'grid',
              'border': '1px solid #000',
              'grid-gap': '8px 8px',
              'background-color':'#fffff9',
            'grid-template-columns': '1fr 1fr'}

def create_grid(
        component_array,num_columns=2,
        column_width_percents=None,
        additional_grid_properties_dict=None,
        wrap_in_loading_state=False,
        row_layout=None):
    gs = GRID_STYLE.copy()
    if row_layout is None:
        percents = [str(round(100/num_columns-.001,1))+'%' for _ in range(num_columns)] if column_width_percents is None else [str(c)+'%' for c in column_width_percents]
        perc_string = " ".join(percents)
        gs['grid-template-columns'] = perc_string 
    else:
        gs['grid-template-columns'] = row_layout
    if additional_grid_properties_dict is not None:
        for k in additional_grid_properties_dict.keys():
            gs[k] = additional_grid_properties_dict[k]           
    
    div_children = []
    for c in component_array:
        if type(c)==str:
            div_children.append(GridItem(c).html)
        elif hasattr(c,'html'):
            div_children.append(c.html)
        else:
            div_children.append(c)
    if wrap_in_loading_state:
        g = dcc.Loading(html.Div(div_children,style=gs),type='cube')
    else:
        g = html.Div(div_children,style=gs)
    return g

class GridItem():
    def __init__(self,child,html_id=None):
        self.child = child
        self.html_id = html_id
    @property
    def html(self):
        if self.html_id is not None:
            return html.Div(children=self.child,className='grid-item',id=self.html_id)
        else:
            return html.Div(children=self.child,className='grid-item')


In [34]:
#********** define useful css styles ***********************
borderline = 'none' #'solid'
button_style={
    'line-height': '40px',
    'borderWidth': '1px',
    'borderStyle': borderline,
    'borderRadius': '1px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}
button_style_no_border={
    'line-height': '40px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}

blue_button_style={
    'line-height': '40px',
    'textAlign': 'center',
    'background-color':'#A9D0F5',#ed4e4e
    'vertical-align':'middle',
}

border_style={
    'line-height': '40px',
    'border':borderline + ' #000',
    'textAlign': 'center',
    'vertical-align':'middle',
}

table_like = {
    'display':'table',
    'width': '100%'
}

# define h4_like because h4 does not play well with dash_table
h4_like = { 
    'display': 'table-cell',
    'textAlign' : 'center',
    'vertical-align' : 'middle',
    'font-size' : '16px',
    'font-weight': 'bold',
    'width': '100%',
    'color':'#22aaff'
}


cte_title_style = { 
    'textAlign' : 'center',
    'vertical-align' : 'middle',
    'font-weight': 'bold',
    'color':'#22aaff'
}

DEFAULT_TIMEZONE = 'US/Eastern'


In [35]:
# ************************* define useful factory methods *****************

def parse_contents(contents):
    '''
    app.layout contains a dash_core_component object (dcc.Store(id='df_memory')), 
      that holds the last DataFrame that has been displayed. 
      This method turns the contents of that dash_core_component.Store object into
      a DataFrame.
      
    :param contents: the contents of dash_core_component.Store with id = 'df_memory'
    :returns pandas DataFrame of those contents
    '''
    c = contents.split(",")[1]
    c_decoded = base64.b64decode(c)
    c_sio = io.StringIO(c_decoded.decode('utf-8'))
    df = pd.read_csv(c_sio)
    # create a date column if there is not one, and there is a timestamp column instead
    cols = df.columns.values
    cols_lower = [c.lower() for c in cols] 
    if 'date' not in cols_lower and 'timestamp' in cols_lower:
        date_col_index = cols_lower.index('timestamp')
        # make date column
        def _extract_dt(t):
            y = int(t[0:4])
            mon = int(t[5:7])
            day = int(t[8:10])
            hour = int(t[11:13])
            minute = int(t[14:16])
            return datetime.datetime(y,mon,day,hour,minute,tzinfo=pytz.timezone(DEFAULT_TIMEZONE))
        # create date
        df['date'] = df.iloc[:,date_col_index].apply(_extract_dt)
    return df

def make_df(dict_df):
    if type(dict_df)==list:
        if type(dict_df[0])==list:
            dict_df = dict_df[0]
        return pd.DataFrame(dict_df,columns=dict_df[0].keys())
    else:
        return pd.DataFrame(dict_df,columns=dict_df.keys())

class BadColumnsException(Exception):
    def __init__(self,*args,**kwargs):
        Exception.__init__(self,*args,**kwargs)



def create_dt_div(dtable_id,df_in=None,
                  columns_to_display=None,
                  editable_columns_in=None,
                  title='Dash Table',logger=None,
                  title_style=None):
    '''
    Create an instance of dash_table.DataTable, wrapped in an dash_html_components.Div
    
    :param dtable_id: The id for your DataTable
    :param df_in:     The pandas DataFrame that is the source of your DataTable (Default = None)
                        If None, then the DashTable will be created without any data, and await for its
                        data from a dash_html_components or dash_core_components instance.
    :param columns_to_display:    A list of column names which are in df_in.  (Default = None)
                                    If None, then the DashTable will display all columns in the DataFrame that
                                    it receives via df_in or via a callback.  However, the column
                                    order that is displayed can only be guaranteed using this parameter.
    :param editable_columns_in:    A list of column names that contain "modifiable" cells. ( Default = None)
    :param title:    The title of the DataFrame.  (Default = Dash Table)
    :param logger:
    :param title_style: The css style of the title. Default is dgrid_components.h4_like.
    '''
    # create logger 
    lg = init_root_logger() if logger is None else logger
    
    lg.debug(f'{dtable_id} entering create_dt_div')
    
    # create list that 
    editable_columns = [] if editable_columns_in is None else editable_columns_in
    datatable_id = dtable_id
    dt = dash_table.DataTable(
        page_current= 0,
        page_size= 100,
        filter_action='none', # 'fe',
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        style_cell_conditional=[
            {
                'if': {'column_id': c},
                'textAlign': 'left',
            } for c in ['symbol', 'underlying']
        ],

        style_as_list_view=False,
        style_table={
            'maxHeight':'450px','overflowX': 'scroll','overflowY':'scroll'
#             'height':'15','overflowX': 'scroll','overflowY':'scroll'
        } ,
        editable=True,
        css=[{"selector": "table", "rule": "width: 100%;"}],
        id=datatable_id
    )
    if df_in is None:
        df = pd.DataFrame({'no_data':[]})
    else:
        df = df_in.copy()
        if columns_to_display is not None:
            if any([c not in df.columns.values for c in columns_to_display]):
                m = f'{columns_to_display} are missing from input data. Your input Csv'
                raise BadColumnsException(m)           
            df = df[columns_to_display]
            
    dt.data=df.to_dict('rows')
    dt.columns=[{"name": i, "id": i,'editable': True if i in editable_columns else False} for i in df.columns.values]                    
    s = h4_like if title_style is None else title_style
    child_div = html.Div([html.Div(html.Div(title,style=s),style=table_like),dt])
    lg.debug(f'{dtable_id} exiting create_dt_div')
    return child_div


In [36]:
def create_xy_graph():
    pass

In [37]:
def recursive_grid_layout(app_component_list,current_component_index,gtcl,layout_components,wrap_in_loading_state=False):
    # loop through the gtcl, and assign components to grids
    for grid_template_columns in gtcl:
        if type(grid_template_columns)==list:
            layout_components.append(recursive_grid_layout(
                app_component_list,current_component_index, grid_template_columns, 
#                 layout_components,wrap_in_loading_state=True))
                layout_components,wrap_in_loading_state=False))
            continue
        # if this grid_template_columns item is NOT a list, process it normally
        sub_list_grid_components = []
        num_of_components_in_sublist = len(grid_template_columns.split(' '))
        for _ in range(num_of_components_in_sublist):
            # get the current component
            layout_ac = app_component_list[current_component_index]
            # add either the component, or it's html property to the sublist
            if hasattr(layout_ac, 'html'):
                layout_ac = layout_ac.html
            sub_list_grid_components.append(layout_ac)
            current_component_index +=1
        new_grid = create_grid(sub_list_grid_components, 
                        additional_grid_properties_dict={'grid-template-columns':grid_template_columns},
                        wrap_in_loading_state=wrap_in_loading_state)
        layout_components.append(new_grid)

def make_layout(comp_list,grid_template_columns_list):
    # for horizontal default
#     default_gtcl = [' '.join(['1fr' for _ in range(len(comp_list))])]
    # for vertical default
    default_gtcl = ['1fr' for _ in range(len(comp_list))]
    gtcl = default_gtcl if grid_template_columns_list is None else grid_template_columns_list
    
    # populate layout_components using recursive algo
    layout_list = []    
    recursive_grid_layout(comp_list,0,gtcl,layout_list)
    return    layout_list

In [38]:
class_converters = {
    dcc.Checklist:lambda v:v,
    dcc.DatePickerRange:lambda v:v,
    dcc.DatePickerSingle:lambda v:v,
    dcc.Dropdown:lambda v:v,
    dcc.Input:lambda v:v,
    dcc.Markdown:lambda v:v,
    dcc.RadioItems:lambda v:v,
    dcc.RangeSlider:lambda v:v,
    dcc.Slider:lambda v:v,
    dcc.Store:lambda v:v,
    dcc.Textarea:lambda v:v,
    dcc.Upload:lambda v:v,
}

html_members = [t[1] for t in inspect.getmembers(html)]
dcc_members = [t[1] for t in inspect.getmembers(dcc)]
all_members = html_members + dcc_members

class DashLink():
    def __init__(self,in_tuple_list, out_tuple_list,io_callback=None,state_tuple_list= None):
        _in_tl = [(k.id if type(k) in all_members else k,v) for k,v in in_tuple_list]
        _out_tl = [(k.id if type(k) in all_members else k,v) for k,v in out_tuple_list]
        self.output_table_names = _out_tl
        
        self.inputs = [Input(k,v) for k,v in _in_tl]
        self.outputs = [Output(k,v) for k,v in _out_tl]
        
        self.states = [] 
        if state_tuple_list is not None:
            _state_tl = [(k.id if type(k) in all_members else k,v) for k,v in state_tuple_list]
            self.states = [State(k,v) for k,v in _state_tl]
        
        self.io_callback = lambda input_list:input_list[0] 
        if io_callback is not None:
            self.io_callback = io_callback
                       
    def callback(self,theapp):
        @theapp.callback(
            self.outputs,
            self.inputs,
            self.states
            )
        def execute_callback(*inputs_and_states):
            l = list(inputs_and_states)
            ret = self.io_callback(l)
            return ret if type(ret) is list else [ret]
        return execute_callback
        
def makeapp(comp_list,dlink_list,grid_layout_list=None):
    app = dash.Dash()
    layout_components = make_layout(comp_list,grid_layout_list)
    grid_html = html.Div(layout_components,style={'margin-left':'10px','margin-right':'10px'})
    app.layout=html.Div(grid_html)
    for dlink in dlink_list:
        dlink.callback(app)
    return app

In [39]:
# !jupyter nbconvert --to script easycomp.ipynb

In [40]:
# header markdown div
# file input: 
# editable table showing table names, and aliases

# area for building sql
# main table alias
# add join statements
# add where statements
# add groupby statements
# add orderby statements

# area for putting sql statements into cte

# run cte

# table display of dataframe

# build graph area

In [41]:
class SqlExpression():
    """
    Build and sql expresssion that can also be fed into a CTE.
    Example:
    
    """
    def __init__(self,main_table_name,main_table_alias=None):
        self.main_table_name = main_table_name
        self.main_table_alias = main_table_name if main_table_alias is None else main_table_alias
        self.join_expressions = []
        self.where_clause_list = []
        self.regular_col_list = []
        self.aggregate_col_list = []
        self.sort_order_list = []    
    
    def get_main_table(self):
        return self.main_table_name
    
    def get_main_table_alias(self):
        return self.main_table_alias

    def add_join_expression(self,join_expression):
        self.join_expressions.append(join_expression)

    def add_where_clause(self,subject, verb,predicate):
        wp = predicate.replace('%','%%') if (type(predicate) == str) else predicate
        wv = f" {verb} "
        ws = f"{subject} "
        where_expression = ws + wv + wp
        self.where_clause_list.append(where_expression)

    def add_regular_display_cols(self,col_list):
        self.regular_col_list.extend(col_list)
        
    def add_aggregate_display_cols(self,agg_expression_list):
        self.aggregate_col_list.extend(agg_expression_list)

    def add_sort_cols(self,col_list):
        self.sort_order_list.extend(col_list)
    
    def make_expression(self):
        join_str = '' if len(self.join_expressions)<1 else 'join ' + ' join '.join(self.join_expressions)
        where_str = '' if len(self.where_clause_list)<1 else 'where ' + ' and '.join(self.where_clause_list)
        reg_col_str = '*' if len(self.regular_col_list)<1 else ','.join(self.regular_col_list)
        agg_col_str = ','.join(self.aggregate_col_list)
        all_col_str = reg_col_str + ('' if len(agg_col_str)<1 else ',' + agg_col_str)
        group_by_str = '' if len(agg_col_str)<1 else f'group by {reg_col_str}'
        order_by_str = '' if len(self.sort_order_list) < 1 else 'order by ' + ','.join(self.sort_order_list)
        expression = f"""
        select {all_col_str} 
        from {self.main_table_name} {self.main_table_alias}
        {join_str}
        {where_str}
        {group_by_str}
        {order_by_str}
        """
        return expression
    
class CteBuilder():
    def __init__(self,sql_expression_list,limit=20):
        self.sql_expression_list = sql_expression_list
        self.limit = limit
        
    def make_sql(self):
        sl = self.sql_expression_list
        sle = [s.make_expression() for s in sl]
        full_sql = 'with \n'
        full_sql += '\n,'.join([f'f{i} as (' + sle[i] + ')' for i in range(len(sle))])
        last_cte_name = str(len(sle)-1)
        lm = '' if self.limit is None else f' limit {self.limit}'
        full_sql += f"\nselect * from f{last_cte_name} {lm}"
        return full_sql

In [42]:
class DashApp():
    def __init__(self):
        self.all_component_tuples = []
        self.all_dash_links = []
        
    def act_append(self,component_list,css_layout=None):
        cssl = css_layout
        if cssl is None:
            cssl = ' '.join(['1fr' for _ in component_list])
        component_already_exists = False
        for comp in component_list:
            for act in self.all_component_tuples:
                for existing_component in act[0]:
                    if comp.id==existing_component.id:
                        component_already_exists = True
                        break
        if not component_already_exists:
            new_tuple = (component_list,cssl)
            self.all_component_tuples.append(new_tuple)
        else:
            print(f'act_append component {comp.id} already in all_component_tuples')

    def make_component_and_css_lists(self):
        comp_list = []
        css_list = []
        for act in self.all_component_tuples:
            comp_list.extend(act[0])
            css_list.append(act[1])
        return comp_list,css_list

    def adl_append(self,dashlink):
        link_already_in_list = False
        for otn in dashlink.output_table_names:
            for adl in self.all_dash_links:
                for adl_otn in adl.output_table_names:
                    if otn == adl_otn:
                        link_already_in_list = True
                        break
        if not link_already_in_list:
            self.all_dash_links.append(dashlink)
        else:
            print(f'adl_append output {otn} already in output in all_dask_links')

            

In [43]:
dap = DashApp()

### title

In [44]:
# set up top title 
title_markedown = """
# Sql Common Table Expression Builder
1. Load csv files to act as your SQL Tables
2. Use the panels below to enter criteria determining:
  * Your main table
  * Your joined tables
  * Your where clauses
  * Your non-aggregate columns to display
  * Your aggregate, if any, columns to display
3. Using the above, build a full SQL Expression
4. Using these SQL Expressions, build a full SQL Common Table Expression (CTE)
5. Click on the execute cte button to run the CTE query and return data
"""
mk1 = dcc.Markdown(title_markedown,id='mk1',style={'color':'black','textAlign': 'left',})
dap.act_append([mk1])


### Create components that allow you to use csv files as SQL Tables
1. create uploader of csv files to use as SQL Tables, and them as Dataframes
2. create a dcc.Store to store a dictionary of the tables by file name 
 * (without the .csv extention)

In [45]:
upload1 = dcc.Upload(
            id='upload1',
            children=html.Div("Choose csv files that represent the tables in your database:"),
            accept = '.csv',
            # Allow multiple files to be uploaded
            multiple=True,
            style=blue_button_style)
dap.act_append([upload1])

dfcsv_init = pd.DataFrame({'table_name':[],'alias':[],'columns':[]})
# create the DashTable div that shows the data
dtcsv = create_dt_div('dtcsv',df_in=dfcsv_init,title=html.H4('Input Tables'),editable_columns_in=['alias'])

def make_dtcsv_table(input_list):
#     print(f'make_dt_csv_table {input_list}')
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        return [dfcsv_init.to_dict('records')]
    file_names = input_list[0]
    file_names = [s.replace('.csv','') for s in file_names]
    aliases = [f't{n}' for n in range(len(file_names))]
    
    # get contents columns so that you can display available columns to query with sql
    contents_list = input_list[1]    
    list_df = [parse_contents(content) for content in contents_list]
    col_lists = [','.join(df.columns.values) for df in list_df]
    dft = pd.DataFrame({'table_name':file_names,'alias':aliases,'columns':col_lists})
    r = [dft.to_dict('records')]
    return r

# make link to update list of tables that are used in sql calls
dtcsv_link = DashLink([(upload1,'filename')],[('dtcsv','data')],make_dtcsv_table,state_tuple_list=[(upload1,'contents')])
dap.adl_append(dtcsv_link)

# create store to hold dataframes
tables_store = dcc.Store(id='tables_store')
def make_tables_store(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('make_tables_store no data',logger)
    # make dataframe dicts
    contents_list = input_list[0]    
    list_df = [parse_contents(content) for content in contents_list]
    
    file_names = input_list[1]
    file_names = [s.replace('.csv','') for s in file_names]
    dict_df = {file_names[i]:list_df[i].to_dict('records') for i in range(len(list_df)) }
    return [dict_df]
tables_store_link = DashLink([(upload1,'contents')],[(tables_store,'data')],make_tables_store,state_tuple_list=[(upload1,'filename')])
dap.adl_append(tables_store_link)

# build a div with the table and the store
dtcsv_cube = dcc.Loading(html.Div([dtcsv,tables_store]),type='cube',id='dtcsv_cube_loading')
dap.act_append([dtcsv_cube])


2020-02-27 19:28:42,737 - root - DEBUG - dtcsv entering create_dt_div
2020-02-27 19:28:42,739 - root - DEBUG - dtcsv exiting create_dt_div


### main table selection dropdowns

In [46]:
# select main table
main_dropdown = dcc.Dropdown(id='main_dropdown')
main_dropdown_div = html.Div(['Select a main table from the dropdown:'],id='main_dropdown_div')
dap.act_append([main_dropdown_div,main_dropdown])

def populate_main_dropdown_options(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('populate_main_dropdown_options no data',logger)
    dict_df = input_list[0]
    if len(dict_df)<1:
        stop_callback('populate_main_dropdown_options no data',logger)
    df_tables = make_df(dict_df)
    table_names = df_tables.table_name.values
    ret = [{'label':tn,'value':tn} for tn in table_names]
    return [ret]
main_dropdown_link = DashLink(
    [('dtcsv','data')],[(main_dropdown,'options')],populate_main_dropdown_options)
dap.adl_append(main_dropdown_link)

main_table_store = dcc.Store(id='main_table_store',data=[])
dap.act_append([main_table_store])
def update_main_table_store(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('update_main_table_store no data',logger)
    main_table_name = input_list[0]
    return [main_table_name]
main_table_store_link = DashLink([(main_dropdown,'value')],[(main_table_store,'data')],
                                update_main_table_store)
dap.adl_append(main_table_store_link)


### create left and right join table selections

In [47]:
# define dcc component, div and DashLink for selecting left table
join_left_radioitems = dcc.RadioItems(id='join_left_radioitems',labelStyle={"display":"block","vertical-align":"middle"})
join_left_radioitems_div = html.Div([html.Div('Select left join table:'),html.Div([join_left_radioitems])],id='join_left_radioitems_div')
def populate_join_left_radio_options(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('populate_join_radio_options no data',logger)
    main_table_name = input_list[0]
    all_table_names = [d['value'] for d in input_list[1]]
#     join_table_names = [jt for jt in all_table_names if jt != main_table_name]
    join_table_names = all_table_names
    ret = [{'label':tn,'value':tn} for tn in join_table_names] #+ [{}]
    return [ret]
join_left_radioitems_link = DashLink(
    [(main_dropdown,'value')],[(join_left_radioitems,'options')],
    populate_join_left_radio_options,state_tuple_list=[(main_dropdown,'options')])
dap.adl_append(join_left_radioitems_link)

# define dcc component, div and DashLink for selecting right table
join_right_radioitems = dcc.RadioItems(id='join_right_radioitems',labelStyle={"display":"block","vertical-align":"middle"})
join_right_radioitems_div = html.Div([html.Div('Select right join table:'),html.Div([join_right_radioitems])],id='join_right_radioitems_div')
def populate_join_right_radio_options(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('populate_join_radio_options no data',logger)
    main_table_name = input_list[0]
    all_table_names = [d['value'] for d in input_list[1]]
    join_table_names = [jt for jt in all_table_names if jt != main_table_name]
    ret = [{'label':tn,'value':tn} for tn in join_table_names] #+ [{}]
    return [ret]
join_right_radioitems_link = DashLink(
    [(join_left_radioitems,'value')],[(join_right_radioitems,'options')],
    populate_join_right_radio_options,state_tuple_list=[(join_left_radioitems,'options')])
dap.adl_append(join_right_radioitems_link)
dap.act_append([join_left_radioitems_div,join_right_radioitems_div])

### Define 3 sets of columns to use for left and right join

In [48]:
# define dcc component, div and DashLink for selecting left table columns
def populate_join_columns_dropdown_options(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('populate_join_columns_dropdown_options no left table_name data',logger)
    table_name = input_list[0]
    dict_df = input_list[1]
    if len(dict_df)<1:
        stop_callback('populate_join_columns_dropdown_options no dataframe data',logger)
    df_tables = make_df(dict_df)
    df_tables_this_table = df_tables[df_tables.table_name==table_name]
    if (len(df_tables_this_table)<1):
        stop_callback(f'populate_join_columns_dropdown_options no data for table {table_name}',logger)
    columns = str(df_tables_this_table.iloc[0]['columns']).split(',')
    ret = [{'label':c,'value':c} for c in columns]
    return [ret]

class ColumnsDropdown():
    def __init__(self,id_base,join_left_radioitems,join_right_radioitems):
        # define left
        self.verb_options = [{'label':v,'value':v} for v in ['=','<','>','<=','>=','like','ilike']]
        self.join_left_columns_dropdown = dcc.Dropdown(id=f'join_left_columns_dropdown_{id_base}')
        self.join_left_columns_dropdown_div = html.Div([html.Div('Select left join column:'),html.Div([self.join_left_columns_dropdown])],
                                                       id=f'join_left_columns_dropdown_div_{id_base}')
        self.join_left_columns_dropdown_link = DashLink(
            [(join_left_radioitems,'value')],[(self.join_left_columns_dropdown,'options')],
            populate_join_columns_dropdown_options,state_tuple_list=[('dtcsv','data')])
        
        # define right
        self.join_right_columns_dropdown = dcc.Dropdown(id=f'join_right_columns_dropdown_{id_base}')
        self.join_right_columns_dropdown_div = html.Div([html.Div('Select right join column:'),html.Div([self.join_right_columns_dropdown])],
                                                       id=f'join_right_columns_dropdown_div_{id_base}')
        self.join_right_columns_dropdown_link = DashLink(
            [(join_right_radioitems,'value')],[(self.join_right_columns_dropdown,'options')],
            populate_join_columns_dropdown_options,state_tuple_list=[('dtcsv','data')])
        
        # define dcc component, div and DashLink for selecting join verb
        self.join_verb_dropdown = dcc.Dropdown(id=f'join_verb_dropdown_{id_base}',value='=',options=self.verb_options,placeholder="select verb")
        self.join_verb_dropdown_div = html.Div([html.Div(['Select a join verb:']),html.Div(self.join_verb_dropdown)],id=f'join_verb_dropdown_div_{id_base}')
        self.div = html.Div(children=[
            self.join_left_columns_dropdown_div,
            self.join_verb_dropdown_div,
            self.join_right_columns_dropdown_div
        ])
        self.links = [self.join_left_columns_dropdown_link,
                     self.join_right_columns_dropdown_link]

# create a button that create the join sql from the dropdowns
cd1 = ColumnsDropdown('cd1',join_left_radioitems,join_right_radioitems)
cd2 = ColumnsDropdown('cd2',join_left_radioitems,join_right_radioitems)
cd3 = ColumnsDropdown('cd3',join_left_radioitems,join_right_radioitems)
for cd in [cd1,cd2,cd3]:
    for lnk in cd.links:
        dap.adl_append(lnk)
        div_list = [
            cd.join_left_columns_dropdown_div,
            cd.join_verb_dropdown_div,
            cd.join_right_columns_dropdown_div]
        dap.act_append(div_list)


act_append component join_right_columns_dropdown_div_cd1 already in all_component_tuples
act_append component join_right_columns_dropdown_div_cd2 already in all_component_tuples
act_append component join_right_columns_dropdown_div_cd3 already in all_component_tuples


### create the div tha holds the join statements, and get's updated via the join_add_button

In [49]:
join_add_button = html.Button('Add join', id='join_add_button')
dap.act_append([join_add_button])

sqlin = dcc.Textarea(id='sqlin')
dap.act_append([sqlin])

join_store = dcc.Store(id='join_store',data=[])
dap.act_append([join_store])

def update_join_store(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('update_join_store no data',logger)
    join_list = input_list[1]
    left_table = input_list[2]
    if any([v is None for v in [left_table,join_list]]):
        stop_callback('update_join_store join info missing',logger)
    right_table = input_list[3]
    join_text = ''
    if input_list[4] is  None:
        stop_callback('update_join_store no column info specified',logger)
    join_text = f"join {left_table} on "
    for i in range(3): 
        if input_list[4+i*3] is not None:
            next_join_clause = f"{left_table}.{input_list[4+i*3]} {input_list[5+i*3]} {left_table}.{input_list[6+i*3]}"
            if i>0:
                next_join_clause = ' and ' + next_join_clause
            join_text += next_join_clause
                    
    # assemble a new join from radioitems, checklists and dropdowns
    join_list.append(join_text)
    return [join_list]

join_store_link = DashLink([(join_add_button,'n_clicks')],[(join_store,'data')],
                          update_join_store,
                          state_tuple_list=[
                              (join_store,'data'),
                              (join_left_radioitems,'value'),
                              (join_right_radioitems,'value'),
                              (cd1.join_left_columns_dropdown,'value'),
                              (cd1.join_verb_dropdown,'value'),
                              (cd1.join_right_columns_dropdown,'value'),
                              (cd2.join_left_columns_dropdown,'value'),
                              (cd2.join_verb_dropdown,'value'),
                              (cd2.join_right_columns_dropdown,'value'),
                              (cd3.join_left_columns_dropdown,'value'),
                              (cd3.join_verb_dropdown,'value'),
                              (cd3.join_right_columns_dropdown,'value')
                          ])
dap.adl_append(join_store_link)

def update_sqlin(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('update_sqldiv no data',logger)
    join_statements = input_list[0] 
    join_text = '\n'.join(join_statements)
    return [join_text]
sqldiv_update_link = DashLink([(join_store,'data')],[(sqlin,'value')],update_sqlin)
dap.adl_append(sqldiv_update_link)


### create columns to display

### create where clause builders

### Define sql_run_button, and div that shows sql results

In [50]:
# define running sql
sql_run_button = html.Button('Execute SQL:',id='sql_run_button')
dap.act_append([sql_run_button])
sqldiv = html.Div(children=[],id='sqldiv')
sqlloading = dcc.Loading(sqldiv,type='cube',id='sqlloading')
dap.act_append([sqlloading])

def make_sqldiv(input_list):
    if input_list is None or len(input_list)<3 or input_list[1] is None:
        stop_callback('make_sqldiv no data',logger)
    sql = input_list[1]
    dict_df = input_list[2]
    if dict_df is None:
        stop_callback('make_sqldiv no csv data selected',logger)
    for k in dict_df.keys():
        globals()[k] = make_df(dict_df[k])
    df_sql_results = sqldf(sql,globals())
    dt_sql_results = create_dt_div('dtsql',df_in=df_sql_results,title=html.H4('Input Tables'))
    return [dt_sql_results]
sqlin_link = DashLink([(sql_run_button,'n_clicks')],[(sqldiv,'children')],make_sqldiv,state_tuple_list=[(sqlin,'value'),(tables_store,'data')])
dap.adl_append(sqlin_link)


In [51]:
# create and run app
components,gtcl = dap.make_component_and_css_lists()
app = makeapp(components,dap.all_dash_links,gtcl)
app.run_server(host='127.0.0.1',port=8500)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-02-27 19:28:42,916 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-02-27 19:28:44,813 - werkzeug - INFO - 127.0.0.1 - - [27/Feb/2020 19:28:44] "GET / HTTP/1.1" 200 -
2020-02-27 19:28:45,016 - werkzeug - INFO - 127.0.0.1 - - [27/Feb/2020 19:28:45] "GET /_dash-layout HTTP/1.1" 200 -
2020-02-27 19:28:45,020 - werkzeug - INFO - 127.0.0.1 - - [27/Feb/2020 19:28:45] "GET /_dash-dependencies HTTP/1.1" 200 -
2020-02-27 19:28:45,199 - root - DEBUG - ****************************** populate_join_radio_options no data ***************************************
2020-02-27 19:28:45,201 - werkzeug - INFO - 127.0.0.1 - - [27/Feb/2020 19:28:45] "POST /_dash-update-component HTTP/1.1" 204 -
2020-02-27 19:28:45,203 - root - DEBUG - ****************************** update_main_table_store no data ***************************************
2020-02-27 19:28:45,206 - werkzeug - INFO - 127.0.0.1 - - [27/Feb/2020 19:28:45] "POST /_dash-update-component HTTP/1.1" 204 -
2020-02-

In [52]:
def ll():
    return [1,2,3],[4,5,6]
a,b = ll()
print(a)
print(b)

[1, 2, 3]
[4, 5, 6]


# End